In [ ]:
# !pip install fuzzywuzzy python-Levenshtein geopandas pandas numpy matplotlib

In [ ]:
# =============================================================================
# DATA LOADING AND PREPROCESSING
# This cell handles the initial loading and preparation of the hurricane tweet data.
# Key steps include:
# 1. Importing necessary libraries for data manipulation, file paths, and time handling.
# 2. Constructing file paths to the GeoJSON data for Hurricanes Francine and Helene.
# 3. Loading the spatial data into GeoDataFrames.
# 4. Standardizing all timestamps to Coordinated Universal Time (UTC).
# 5. Aggregating the data into discrete 4-hour time bins for temporal analysis.
# 6. Creating various time-related columns (Unix timestamps, readable labels) for later use.
# =============================================================================

# Import core libraries
import geopandas as gpd  # Used for working with geospatial data.
import pandas as pd      # Used for data manipulation and analysis in DataFrames.
import os                # Provides a way of using operating system dependent functionality, like file paths.
from datetime import datetime, timezone # Used for handling date and time objects.

# --- 1. Load GeoJSON files ---
# Get the parent directory of the current working directory to build relative paths.
# This makes the script more portable as it doesn't rely on a hardcoded absolute path.
local_path = os.path.dirname(os.getcwd())

# Define the relative paths to the GeoJSON files for each hurricane.
francine_dir = r"\data\geojson\francine.geojson"
helene_dir = r"\data\geojson\helene.geojson"

# Combine the base path and relative directory to create full, absolute paths to the files.
francine_path = f"{local_path}{francine_dir}"
helene_path = f"{local_path}{helene_dir}"

# --- 2. Load data into GeoDataFrames ---
# A GeoDataFrame is a pandas DataFrame with a special 'geometry' column that allows for spatial operations.
francine_gdf = gpd.read_file(francine_path)
helene_gdf = gpd.read_file(helene_path)

# --- 3. Standardize timestamps to UTC ---
# Convert the original 'time' column into a pandas datetime object.
# Setting `utc=True` ensures all timestamps are in a single, unambiguous timezone (UTC).
# This is crucial for accurate temporal comparisons and binning.
francine_gdf['timestamp'] = pd.to_datetime(francine_gdf['time'], utc=True)
helene_gdf['timestamp'] = pd.to_datetime(helene_gdf['time'], utc=True)

# --- 4. Group data into 4-hour time bins ---
# The `dt.floor('4h')` function rounds each timestamp *down* to the nearest 4-hour interval.
# For example, 09:35 becomes 08:00, 15:59 becomes 12:00. This aggregates tweets into discrete time windows.
francine_gdf['time_bin'] = francine_gdf['timestamp'].dt.floor('4h')
helene_gdf['time_bin'] = helene_gdf['timestamp'].dt.floor('4h')

# --- 5. Create Unix timestamps and lookup dictionaries ---
# Convert the binned datetime objects into Unix timestamps (as an integer).
# The `// 1000` division is likely to convert from nanoseconds or microseconds to seconds, a more standard Unix format.
francine_gdf['unix_timestamp'] = francine_gdf['time_bin'].astype('int64') // 1000
helene_gdf['unix_timestamp'] = helene_gdf['time_bin'].astype('int64') // 1000

# Create dictionaries to map the numeric Unix timestamp back to its original datetime object.
# This provides a quick way to retrieve the readable time bin later in the script without recalculating it.
helene_timestamp_dict = dict(zip(helene_gdf['unix_timestamp'], helene_gdf['time_bin']))
francine_timestamp_dict = dict(zip(francine_gdf['unix_timestamp'], francine_gdf['time_bin']))

# --- 6. Create readable labels for file naming ---
# The `dt.strftime` function formats the datetime object into a specific string format.
# Here, '%Y%m%d_%H%M' creates a clean, sortable label like '20240926_0800', which is ideal for filenames.
francine_gdf['bin_label'] = francine_gdf['time_bin'].dt.strftime('%Y%m%d_%H%M')
helene_gdf['bin_label'] = helene_gdf['time_bin'].dt.strftime('%Y%m%d_%H%M')

In [ ]:
# Load reference shapefiles
from fuzzywuzzy import fuzz, process
import re

states_dir = r"\data\shape_files\cb_2023_us_state_20m.shp"
counties_dir = r"\data\shape_files\cb_2023_us_county_20m.shp"
cities_dir = r"\data\shape_files\US_Cities.shp"
states_path = f"{local_path}{states_dir}"
counties_path = f"{local_path}{counties_dir}"
cities_path = f"{local_path}{cities_dir}"


# Load spatial reference data
states_gdf = gpd.read_file(states_path)
counties_gdf = gpd.read_file(counties_path)
cities_gdf = gpd.read_file(cities_path)

# =============================================================================
# MULTI-LEVEL GEOGRAPHIC MATCHING (SINGLE BEST PER ENTITY: STATE>COUNTY>CITY)
# with conservative guards against false positives (e.g., FRANCINE→FRANKLIN)
# =============================================================================

from fuzzywuzzy import fuzz, process
import re

# --- Safeguards ---------------------------------------------------------------
BLOCKLIST_EXACT = {
    # hurricane/event terms and generic weather tokens to ignore unless exact place
    'FRANCINE', 'HELENE', 'HURRICANE', 'STORM', 'TROPICAL', 'TS', 'CYCLONE',
    'FLOOD', 'RAIN', 'WIND', 'SURGE', 'LANDFALL', 'EYE', 'TRACK', 'BAND'
}

def is_blocklisted(entity: str) -> bool:
    e = (entity or '').strip().upper()
    return e in BLOCKLIST_EXACT

def preprocess_place_name(name):
    """Standardize place names for better matching."""
    if pd.isna(name) or name == 'NAN':
        return None
    name = str(name).upper().strip()
    name = re.sub(r'\bST\.?\b', 'SAINT', name)
    name = re.sub(r'\bMT\.?\b', 'MOUNT', name)
    name = re.sub(r'\bFT\.?\b', 'FORT', name)
    name = re.sub(r'\bN\.?\b', 'NORTH', name)
    name = re.sub(r'\bS\.?\b', 'SOUTH', name)
    name = re.sub(r'\bE\.?\b', 'EAST', name)
    name = re.sub(r'\bW\.?\b', 'WEST', name)
    name = re.sub(r'[^\w\s]', '', name)
    name = re.sub(r'\s+', ' ', name)
    return name.strip()

def parse_gpe_entities(gpe_string):
    """Split the GPE field into potential place names."""
    if not gpe_string or pd.isna(gpe_string) or str(gpe_string).strip() == '':
        return []
    gpe_string = str(gpe_string).strip()
    entities = []
    for part in [p.strip() for p in gpe_string.split(',')]:
        if not part:
            continue
        for sub in re.split(r'[;&|]', part):
            sub = preprocess_place_name(sub)
            if sub and len(sub) > 1:
                entities.append(sub)
    # remove dups
    seen, clean = set(), []
    for e in entities:
        if e not in seen:
            clean.append(e)
            seen.add(e)
    return clean

def create_hierarchical_lookups(states_gdf, counties_gdf, cities_gdf):
    """Build dictionaries for all levels."""
    print("\nCreating hierarchical lookup dictionaries...")

    state_lookup, state_abbrev_to_name, state_name_to_abbrev = {}, {}, {}
    for _, row in states_gdf.iterrows():
        sname = preprocess_place_name(row['NAME'])
        if not sname:
            continue
        state_lookup[sname] = row.geometry
        if 'STUSPS' in row:
            abbr = str(row['STUSPS']).upper()
            state_abbrev_to_name[abbr] = sname
            state_name_to_abbrev[sname] = abbr
            state_lookup[abbr] = row.geometry

    county_lookup, county_by_state = {}, {}
    for _, row in counties_gdf.iterrows():
        cname = preprocess_place_name(row['NAME'])
        sfips = row.get('STATEFP', '')
        if not cname:
            continue
        county_lookup[cname] = row.geometry
        sname = None
        if 'STATE_NAME' in row:
            sname = preprocess_place_name(row['STATE_NAME'])
        else:
            for _, srow in states_gdf.iterrows():
                if srow.get('STATEFP', '') == sfips:
                    sname = preprocess_place_name(srow['NAME'])
                    break
        if sname:
            county_by_state.setdefault(sname, {})[cname] = row.geometry

    city_lookup, city_by_state = {}, {}
    for _, row in cities_gdf.iterrows():
        cname = preprocess_place_name(row['NAME'])
        stabbr = str(row.get('ST', '')).upper()
        if not cname:
            continue
        city_lookup[cname] = row.geometry
        if stabbr in state_abbrev_to_name:
            sfull = state_abbrev_to_name[stabbr]
            city_by_state.setdefault(sfull, {})[cname] = row.geometry

    return {
        'state_lookup': state_lookup,
        'county_lookup': county_lookup,
        'city_lookup': city_lookup,
        'county_by_state': county_by_state,
        'city_by_state': city_by_state,
        'state_abbrev_to_name': state_abbrev_to_name,
        'state_name_to_abbrev': state_name_to_abbrev
    }

# --- Smart fuzzy matching (with conservative guards) --------------------------
def _best_name_by_ratio(entity, names):
    """Get best candidate name by fuzz.ratio; returns (name, ratio) or (None,0)."""
    if not names:
        return (None, 0)
    match = process.extractOne(entity, names, scorer=fuzz.ratio)
    return (match[0], match[1]) if match else (None, 0)

def fuzzy_match_best(entity, candidates, base_threshold):
    """
    Return (name, geom, score) for the best candidate meeting conservative checks:
      - Blocklisted tokens: only allow EXACT match (no fuzzy).
      - Single-token with len>=6: ONLY exact match (prevents FRANCINE→FRANKLIN).
      - Otherwise require BOTH ratio and token_set_ratio >= threshold.
    """
    if not entity or not candidates:
        return None, None, 0

    # Exact fast-path
    if entity in candidates:
        return entity, candidates[entity], 100

    # Blocklist: refuse fuzzy for these tokens
    if is_blocklisted(entity):
        return None, None, 0

    # Single-token long strings: exact only
    if ' ' not in entity and len(entity) >= 6:
        return None, None, 0

    # Compute best by ratio, then validate with token_set_ratio
    names = list(candidates.keys())
    best_name, ratio_score = _best_name_by_ratio(entity, names)
    if not best_name:
        return None, None, 0

    set_score = fuzz.token_set_ratio(entity, best_name)

    # Dynamic tightening for very short tokens (reduce false hits)
    threshold = base_threshold
    if len(entity) <= 3:
        threshold = max(threshold, 95)
    elif len(entity) <= 5:
        threshold = max(threshold, 90)

    if ratio_score >= threshold and set_score >= threshold:
        return best_name, candidates[best_name], int((ratio_score + set_score) / 2)

    return None, None, 0

def resolve_best_for_entity(entity, lookups, state_context):
    """
    Pick ONE best match with priority: STATE > COUNTY > CITY.
    COUNTY/CITY biased by any known states in state_context.
    """
    st_lu = lookups['state_lookup']
    co_lu = lookups['county_lookup']
    ci_lu = lookups['city_lookup']
    co_by_state = lookups['county_by_state']
    ci_by_state = lookups['city_by_state']

    # 1) STATE (exact or USPS abbrev or conservative fuzzy)
    n, g, sc = fuzzy_match_best(entity, st_lu, 85)  # states: higher threshold
    if sc > 0:
        return ('STATE', n, g, sc)

    # 2) COUNTY (prefer in-state)
    best = (None, None, 0)
    for s in state_context:
        if s in co_by_state:
            n2, g2, sc2 = fuzzy_match_best(entity, co_by_state[s], 85)
            if sc2 > best[2]:
                best = (n2, g2, sc2)
    if best[2] == 0:
        n2, g2, sc2 = fuzzy_match_best(entity, co_lu, 90)  # global county stricter
        if sc2 > best[2]:
            best = (n2, g2, sc2)
    if best[2] > 0:
        return ('COUNTY', best[0], best[1], best[2])

    # 3) CITY (prefer in-state)
    best = (None, None, 0)
    for s in state_context:
        if s in ci_by_state:
            n3, g3, sc3 = fuzzy_match_best(entity, ci_by_state[s], 85)
            if sc3 > best[2]:
                best = (n3, g3, sc3)
    if best[2] == 0:
        n3, g3, sc3 = fuzzy_match_best(entity, ci_lu, 90)  # global city stricter
        if sc3 > best[2]:
            best = (n3, g3, sc3)
    if best[2] > 0:
        return ('CITY', best[0], best[1], best[2])

    return None

def find_all_geographic_matches_single_per_entity(entities, lookups):
    """Return one best match per entity, using STATE>COUNTY>CITY and state context."""
    if not entities:
        return []

    # Build state context from the entities themselves (helps county/city later).
    state_context = set()
    for e in entities:
        n, g, sc = fuzzy_match_best(e, lookups['state_lookup'], 85)
        if sc > 0 and n:
            state_context.add(n)

    results, seen = [], set()
    for e in entities:
        m = resolve_best_for_entity(e, lookups, state_context)
        if not m:
            continue
        key = (m[0], m[1])  # (scale, name)
        if key not in seen:
            results.append(m)
            seen.add(key)
    return results

def multi_level_assign_scale_levels(row, lookups):
    """Return list of matches [(scale,name,geom,score), ...] for this tweet."""
    gpe = str(row.get('GPE', '')).strip()
    fac = str(row.get('FAC', '')).strip()
    matches = []

    entities = parse_gpe_entities(gpe)
    if entities:
        geo_matches = find_all_geographic_matches_single_per_entity(entities, lookups)
        matches.extend(geo_matches)

    if fac and fac not in ['nan', 'NAN', '']:
        matches.append(('FACILITY', preprocess_place_name(fac), row.geometry, 100))

    if not matches:
        matches.append(('UNMATCHED', None, row.geometry, 0))
    return matches

def expand_tweets_by_matches(gdf, lookups, dataset_name):
    print(f"\nExpanding {dataset_name} tweets by geographic matches...")
    expanded_rows = []
    for i, row in gdf.iterrows():
        if i % 100 == 0:
            print(i)
        for scale, name, geom, score in multi_level_assign_scale_levels(row, lookups):
            new_row = row.copy()
            new_row['scale_level'] = scale
            new_row['matched_name'] = name
            new_row['matched_geom'] = geom
            new_row['match_score'] = score
            new_row['original_index'] = i
            expanded_rows.append(new_row)

    expanded_gdf = gpd.GeoDataFrame(expanded_rows, crs=gdf.crs)
    print("  Sample multi-level matches:")
    multi = expanded_gdf.groupby('original_index').size()
    idxs = multi[multi > 1].head(5).index
    for j in idxs:
        t = expanded_gdf[expanded_gdf['original_index'] == j]
        gpe = t.iloc[0]['GPE']
        summ = ', '.join([f"{r['scale_level']}:{r['matched_name']}" for _, r in t.iterrows()])
        print(f"    '{gpe}' → {summ}")
    return expanded_gdf

# =============================================================================
# EXECUTE MULTI-LEVEL FUZZY MATCHING
# =============================================================================
print("\n" + "="*60)
print("MULTI-LEVEL GEOGRAPHIC MATCHING (STATE>COUNTY>CITY, conservative)")
print("="*60)

lookups = create_hierarchical_lookups(states_gdf, counties_gdf, cities_gdf)
francine_gdf = expand_tweets_by_matches(francine_gdf, lookups, "FRANCINE")
helene_gdf   = expand_tweets_by_matches(helene_gdf, lookups, "HELENE")

print("\n" + "="*60)
print("MULTI-LEVEL FUZZY MATCHING COMPLETE ✓")
print("="*60)
print("\nGuards active: blocklist, exact-only for long single tokens, dual-score thresholds.")


In [ ]:
 # Group tweets by 4-hour intervals and scale level
# Using unix_timestamp for unambiguous temporal grouping

# Alternative approach:
francine_interval_counts = francine_gdf.groupby(['unix_timestamp', 'scale_level', 'matched_name']).agg({
    'matched_geom': 'first'
}).reset_index()

# Add count column separately
count_series = francine_gdf.groupby(['unix_timestamp', 'scale_level', 'matched_name']).size()
francine_interval_counts['count'] = count_series.values

# Same for Helene
helene_interval_counts = helene_gdf.groupby(['unix_timestamp', 'scale_level', 'matched_name']).agg({
    'matched_geom': 'first'
}).reset_index()
count_series = helene_gdf.groupby(['unix_timestamp', 'scale_level', 'matched_name']).size()
helene_interval_counts['count'] = count_series.values

# Sort by timestamp to ensure chronological order
francine_interval_counts = francine_interval_counts.sort_values('unix_timestamp')
helene_interval_counts = helene_interval_counts.sort_values('unix_timestamp')

# Calculate cumulative counts
francine_interval_counts['cumulative_count'] = francine_interval_counts.groupby(['scale_level', 'matched_name'])['count'].cumsum()
helene_interval_counts['cumulative_count'] = helene_interval_counts.groupby(['scale_level', 'matched_name'])['count'].cumsum()

# Get unique time bins for iteration
francine_time_bins = sorted(francine_gdf['unix_timestamp'].unique())
helene_time_bins = sorted(helene_gdf['unix_timestamp'].unique())



In [ ]:
import numpy as np
import rasterio
from rasterio.transform import from_bounds

# ==============================================================================
# STEP 1: DEFINE MASTER GRID CANVAS
# ==============================================================================

# Configuration
TARGET_CRS = 'EPSG:3857'  # Web Mercator
CELL_SIZE_M = 1000  # 5 km in meters

print("=" * 60)
print("STEP 1: CREATING MASTER GRID CANVAS")
print("=" * 60)

# Project both datasets to target CRS
print(f"\nProjecting datasets to {TARGET_CRS}...")
francine_proj = francine_gdf.to_crs(TARGET_CRS)
helene_proj = helene_gdf.to_crs(TARGET_CRS)

# Also project reference geometries
print("Projecting reference geometries...")
states_proj = states_gdf.to_crs(TARGET_CRS)
counties_proj = counties_gdf.to_crs(TARGET_CRS)
cities_proj = cities_gdf.to_crs(TARGET_CRS)
# Calculate combined extent from both hurricanes"
print("\nCalculating master extent...")
francine_bounds = francine_proj.total_bounds
helene_bounds = helene_proj.total_bounds

# Get union of both bounding boxes
minx = min(francine_bounds[0], helene_bounds[0])
miny = min(francine_bounds[1], helene_bounds[1])
maxx = max(francine_bounds[2], helene_bounds[2])
maxy = max(francine_bounds[3], helene_bounds[3])
#
# print(f"  Master bounds (EPSG:3857):")
# print(f"    minx: {minx:,.2f}")
# print(f"    miny: {miny:,.2f}")
# print(f"    maxx: {maxx:,.2f}")
# print(f"    maxy: {maxy:,.2f}")

# Calculate grid dimensions
width = int(np.ceil((maxx - minx) / CELL_SIZE_M))
height = int(np.ceil((maxy - miny) / CELL_SIZE_M))

print(f"\nGrid Configuration:")
print(f"  Cell size: {CELL_SIZE_M:,} meters ({CELL_SIZE_M/1000} km)")
print(f"  Grid dimensions: {width} x {height} cells")
print(f"  Total cells: {width * height:,}")

# Create master transform
master_transform = from_bounds(minx, miny, maxx, maxy, width, height)

print(f"\nMaster Transform:")
print(f"  {master_transform}")

# Calculate actual coverage area
area_km2 = (width * height * CELL_SIZE_M * CELL_SIZE_M) / 1_000_000
print(f"\nCoverage area: {area_km2:,.2f} km²")

# Store grid parameters for later use
grid_params = {
    'crs': TARGET_CRS,
    'cell_size': CELL_SIZE_M,
    'width': width,
    'height': height,
    'bounds': (minx, miny, maxx, maxy),
    'transform': master_transform
}

print(f"\n{'=' * 60}")
print("MASTER GRID CANVAS READY ✓")
print(f"{'=' * 60}")

# Update lookup dictionaries with projected geometries
print("\nUpdating geometry lookups with projected coordinates...")
state_lookup_proj = dict(zip(states_proj['NAME'].str.upper(), states_proj.geometry))
county_lookup_proj = dict(zip(counties_proj['NAME'].str.upper(), counties_proj.geometry))
cities_lookup_proj = dict(zip(cities_proj['NAME'].str.upper(), cities_proj.geometry))
# validation_results = validate_city_matching(francine_gdf, helene_gdf, lookups['city_lookup'], lookups['state_lookup'], lookups['county_lookup'])
print("Lookup dictionaries updated with projected geometries ✓")

In [ ]:
import os
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon
# Removed: from KDEpy import FFTKDE (using scipy.stats.gaussian_kde instead)
from rasterio.features import rasterize
from rasterio.features import geometry_mask

# ==============================================================================
# STEP 2: MAIN RASTERIZATION LOOP - TIME ITERATION (UNCHANGED CORE, BONUS ADDED)
# ==============================================================================

# Create output directories
rasters_dir = r"\rasters_output"
output_dir = f"{local_path}{rasters_dir}"
os.makedirs(output_dir, exist_ok=True)


def create_hierarchical_rasters(data, grid_params, time_bin):
    """Create hierarchically weighted rasters with KDE heat map for cities"""
    print(f"    Creating hierarchical raster for time {time_bin}...")

    output_grid = np.zeros((grid_params['height'], grid_params['width']), dtype=np.float32)
    states_to_include = set()

    # 1. States (identify base layers to include)
    state_data = data[data['scale_level'] == 'STATE']
    if len(state_data) > 0:
        states_to_include.update(state_data['matched_name'].unique())

    # 1b. Counties -> parent states
    county_data = data[data['scale_level'] == 'COUNTY']
    for county_name in county_data['matched_name'].unique():
        if county_name in county_lookup_proj:
            county_geom = county_lookup_proj[county_name]
            for state_name, state_geom in state_lookup_proj.items():
                if state_geom.contains(county_geom.centroid):
                    states_to_include.add(state_name)
                    break

    # 1c. Cities -> parent states
    city_data = data[data['scale_level'] == 'CITY']
    for city_name in city_data['matched_name'].unique():
        if city_name in cities_lookup_proj:
            city_geom = cities_lookup_proj[city_name]
            for state_name, state_geom in state_lookup_proj.items():
                if state_geom.contains(city_geom.centroid):
                    states_to_include.add(state_name)
                    break

    # 2. Rasterize states
    for state_name in states_to_include:
        if state_name in state_lookup_proj:
            state_geom = state_lookup_proj[state_name]
            mask = rasterize(
                [(state_geom, 1)],
                out_shape=(grid_params['height'], grid_params['width']),
                transform=grid_params['transform'],
                fill=0, dtype=np.float32, all_touched=True
            )
            if state_name in state_data['matched_name'].values:
                tweet_count = state_data[state_data['matched_name'] == state_name]['count'].sum()
            else:
                tweet_count = 1  # minimal base
            base_value = np.log1p(tweet_count) * 2
            output_grid += mask * base_value

    # 3. Add counties (fill)
    if len(county_data) > 0:
        county_counts = county_data.groupby('matched_name')['count'].sum()
        for county_name, tweet_count in county_counts.items():
            if county_name in county_lookup_proj:
                mask = rasterize(
                    [(county_lookup_proj[county_name], 1)],
                    out_shape=(grid_params['height'], grid_params['width']),
                    transform=grid_params['transform'],
                    fill=0, dtype=np.float32, all_touched=True
                )
                output_grid += mask * np.log1p(tweet_count) * 5

    # 4. Add cities as KDE heat map (UPDATED - uses KDE instead of polygon fill)
    if len(city_data) > 0:
        print(f"      Adding city heat map layer...")
        xy, w = _city_centroids_and_weights(city_data)
        if xy is not None:
            # Use KDE with 10km bandwidth
            city_kde = _evaluate_fftkde_on_grid(xy, w, grid_params, bw_meters=10000.0)
            # Add to output with multiplier for visibility
            output_grid += city_kde * 10  # Multiplier to make KDE visible (KDE max ~0.25, need ~25 to compete with log values)
        else:
            print(f"      No valid city centroids for KDE")

    # 5. Facilities (unchanged)
    facility_data = data[data['scale_level'] == 'FACILITY']
    if len(facility_data) > 0:
        output_grid += create_facility_raster(data, grid_params)

    return output_grid

# ==============================================================================
# BONUS: KDEpy (FFTKDE) city-centroid KDE per-bin and cumulative
# ==============================================================================

def _city_centroids_and_weights(city_data):
    """
    Return (N,2) array of centroid coordinates (already projected) and weights.
    Falls back to .centroid for polygons; passes through points. Skips invalids.
    """
    xs, ys, ws = [], [], []
    if len(city_data) == 0:
        return None, None

    city_counts = city_data.groupby('matched_name')['count'].sum()

    for city_name, w in city_counts.items():
        geom = cities_lookup_proj.get(city_name)
        if geom is None:
            continue
        if isinstance(geom, (Polygon, MultiPolygon)):
            c = geom.centroid
        elif isinstance(geom, Point):
            c = geom
        else:
            c = getattr(geom, 'centroid', None)
            if c is None:
                continue
        xs.append(c.x)
        ys.append(c.y)
        ws.append(float(w))
    if len(xs) == 0:
        return None, None
    xy = np.column_stack([np.array(xs, dtype='float64'),
                          np.array(ys, dtype='float64')])
    w = np.array(ws, dtype='float64')
    return xy, w


def _evaluate_fftkde_on_grid(xy, weights, grid_params, bw_meters=10000.0):
    """
    Create a kernel density heat map like ArcGIS Pro's point density tool.

    Takes city centroid points with tweet counts and creates a smooth density
    surface with the specified search radius (bandwidth).

    Returns float32 array aligned to raster orientation (row 0 = top).
    """
    from scipy.stats import gaussian_kde

    H, W = grid_params['height'], grid_params['width']
    xmin, ymin, xmax, ymax = grid_params['bounds']
    cs = grid_params['cell_size']

    # Build grid cell centers
    xs = xmin + (np.arange(W) + 0.5) * cs
    ys = ymin + (np.arange(H) + 0.5) * cs
    X, Y = np.meshgrid(xs, ys, indexing='xy')
    positions = np.vstack([X.ravel(), Y.ravel()])

    # Prepare data for scipy: shape (2, n_points)
    data = xy.T

    # Calculate bandwidth to achieve desired search radius in meters
    data_std = np.sqrt((np.var(data[0, :]) + np.var(data[1, :])) / 2)
    bw_method = bw_meters / data_std if data_std > 0 else 0.1

    try:
        # Fit gaussian KDE with weights
        kde = gaussian_kde(data, bw_method=bw_method, weights=weights)

        # Evaluate - returns probability density (per square meter)
        z_flat = kde(positions)

        # Convert from density (per m²) to total count in each cell
        # Multiply by: (1) cell area and (2) sum of weights
        cell_area = cs * cs  # square meters
        total_weight = np.sum(weights)
        z_flat = z_flat * cell_area * total_weight

        # Reshape and flip
        z = z_flat.reshape(H, W)
        z = np.flipud(z).astype('float32')

        print(f"      City KDE: min={z.min():.3f}, max={z.max():.3f}, mean={z.mean():.3f}")

        return z

    except Exception as e:
        print(f"      ERROR: City KDE failed: {e}")
        return np.zeros((H, W), dtype=np.float32)

def process_hurricane(hurricane_name, gdf_proj, interval_counts, time_bins, timestamp_dict):
    """
    Process a single hurricane through all time bins
    """
    print(f"\n{'=' * 60}")
    print(f"PROCESSING: {hurricane_name.upper()}")
    print(f"{'=' * 60}\n")

    # hurricane output root
    hurricane_dir = os.path.join(output_dir, hurricane_name.lower())
    os.makedirs(hurricane_dir, exist_ok=True)

    # Main cumulative grid (includes states, counties, cities KDE, facilities)
    cumulative_grid = np.zeros((grid_params['height'], grid_params['width']), dtype=np.float32)

    for idx, time_bin in enumerate(time_bins):
        # Filter this bin
        current_data = interval_counts[interval_counts['unix_timestamp'] == time_bin]

        # Create hierarchical raster (now includes city KDE)
        incremental_grid = create_hierarchical_rasters(current_data, grid_params, time_bin)

        # Update cumulative
        cumulative_grid += incremental_grid

        # Save rasters
        save_raster(incremental_grid, hurricane_dir, hurricane_name, time_bin, 'increment', timestamp_dict)
        save_raster(cumulative_grid, hurricane_dir, hurricane_name, time_bin, 'cumulative', timestamp_dict)

        print(f"  Incremental max:       {np.max(incremental_grid):.3f}")
        print(f"  Cumulative max:        {np.max(cumulative_grid):.3f}")

    print(f"\n{hurricane_name.upper()} processing complete!")
    print(f"Output saved to: {hurricane_dir}")
    return

# ==============================================================================
# PLACEHOLDER FUNCTIONS (UNCHANGED)
# ==============================================================================

def create_facility_raster(data, grid_params):
    """Create KDE raster for facility points with strong hotspot multiplier"""
    print("    [FACILITY] Creating facility raster...")

    facility_grid = np.zeros((grid_params['height'], grid_params['width']), dtype=np.float32)
    facility_data = data[data['scale_level'] == 'FACILITY']
    if len(facility_data) == 0:
        print("      No facility-level tweets in this time bin")
        return facility_grid

    facility_counts = facility_data.groupby('matched_name')['count'].sum()
    sigma_meters = 2 * grid_params['cell_size']
    sigma_pixels = sigma_meters / grid_params['cell_size']
    facility_multiplier = 10

    facilities_processed = 0
    for facility_name, tweet_count in facility_counts.items():
        rows = facility_data[facility_data['matched_name'] == facility_name]
        if len(rows) == 0:
            continue
        facility_point = rows.iloc[0]['matched_geom']
        if hasattr(facility_point, 'x') and hasattr(facility_point, 'y'):
            point_geoseries = gpd.GeoSeries([facility_point], crs='EPSG:4326')
            point_proj = point_geoseries.to_crs(grid_params['crs']).iloc[0]
            px = (point_proj.x - grid_params['bounds'][0]) / grid_params['cell_size']
            py = (grid_params['bounds'][3] - point_proj.y) / grid_params['cell_size']
            if 0 <= px < grid_params['width'] and 0 <= py < grid_params['height']:
                point_grid = np.zeros((grid_params['height'], grid_params['width']), dtype=np.float32)
                point_grid[int(py), int(px)] = tweet_count
                # simple gaussian via pixels (kept as-is)
                from scipy.ndimage import gaussian_filter
                kernel_grid = gaussian_filter(point_grid, sigma=sigma_pixels, mode='constant', cval=0)
                facility_grid += kernel_grid * facility_multiplier
                facilities_processed += 1
            else:
                print(f"      WARNING: Facility '{facility_name}' outside grid bounds")
        else:
            print(f"      WARNING: Invalid geometry for facility '{facility_name}'")

    print(f"      Processed {facilities_processed} facilities with sigma={sigma_pixels:.2f} pixels")
    return facility_grid


def save_raster(grid, output_dir, hurricane_name, time_bin, raster_type, timestamp_dict):
    """Save raster as GeoTIFF with embedded time metadata"""
    type_dir = os.path.join(output_dir, raster_type)
    os.makedirs(type_dir, exist_ok=True)

    time_str = timestamp_dict[time_bin].strftime('%Y%m%d_%H%M%S')
    filename = f"{hurricane_name}_tweets_{time_str}.tif"
    filepath = os.path.join(type_dir, filename)

    with rasterio.open(
        filepath, 'w',
        driver='GTiff',
        height=grid_params['height'],
        width=grid_params['width'],
        count=1,
        dtype=grid.dtype,
        crs=grid_params['crs'],
        transform=grid_params['transform'],
        compress='lzw'
    ) as dst:
        dst.write(grid, 1)

        # *** EMBED TIME METADATA ***
        dst.update_tags(1,
            time_bin=str(time_bin),
            timestamp=time_str,
            datetime_iso=timestamp_dict[time_bin].isoformat()
        )
        dst.set_band_description(1, f"Tweet density - {time_str}")

    print(f"    Saved: {raster_type}/{filename}")


# ==============================================================================
# EXECUTE PROCESSING FOR BOTH HURRICANES
# ==============================================================================

print("\n" + "=" * 60)
print("STARTING RASTERIZATION PROCESS")
print("=" * 60)

process_hurricane('francine', francine_proj, francine_interval_counts, francine_time_bins, francine_timestamp_dict)
process_hurricane('helene', helene_proj, helene_interval_counts, helene_time_bins, helene_timestamp_dict)

print("\n" + "=" * 60)
print("ALL PROCESSING COMPLETE! ✓")
print("=" * 60)


In [ ]:
import rasterio
path = r"C:\Users\colto\Documents\GitHub\Tweet_project\rasters_output\helene\cumulative\helene_tweets_20240927_000000.tif"
with rasterio.open(path) as src:
    tags = src.tags(1)
    print(tags)  # Returns '20240926_000000'

In [ ]:
import arcpy
import os
from datetime import datetime


# Note this is to be inserted into the python command window
# Paths
gdb_path = r"C:\Users\colto\Documents\GitHub\Tweet_project\Tweet_project.gdb"


raster_folder = r"C:\Users\colto\Documents\GitHub\Tweet_project\rasters_output\helene\cumulative"
mosaic_name = "helene_cumulative_mosaic_v4"

# raster_folder = r"C:\Users\colto\Documents\GitHub\Tweet_project\rasters_output\helene\increment"
# mosaic_name = "helene_increment_mosaic_v2"
#
# raster_folder = r"C:\Users\colto\Documents\GitHub\Tweet_project\rasters_output\francine\cumulative"
# mosaic_name = "francine_cumulative_mosaic_v2"
#
# raster_folder = r"C:\Users\colto\Documents\GitHub\Tweet_project\rasters_output\francine\increment"
# mosaic_name = "francine_increment_mosaic_v2"



# Create geodatabase if it doesn't exist
if not arcpy.Exists(gdb_path):
    arcpy.CreateFileGDB_management(os.path.dirname(gdb_path), os.path.basename(gdb_path))

# Create mosaic dataset
mosaic_path = os.path.join(gdb_path, mosaic_name)
if arcpy.Exists(mosaic_path):
    arcpy.Delete_management(mosaic_path)

arcpy.CreateMosaicDataset_management(gdb_path, mosaic_name, "PROJCS['WGS_1984_Web_Mercator_Auxiliary_Sphere',GEOGCS['GCS_WGS_1984',DATUM['D_WGS_1984',SPHEROID['WGS_1984',6378137.0,298.257223563]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]],PROJECTION['Mercator_Auxiliary_Sphere'],PARAMETER['False_Easting',0.0],PARAMETER['False_Northing',0.0],PARAMETER['Central_Meridian',0.0],PARAMETER['Standard_Parallel_1',0.0],PARAMETER['Auxiliary_Sphere_Type',0.0],UNIT['Meter',1.0]]")

print(f"Created mosaic dataset: {mosaic_path}")

# Add rasters to mosaic
arcpy.AddRastersToMosaicDataset_management(
    mosaic_path,
    "Raster Dataset",
    raster_folder,
    filter="*.tif"
)

print("Added rasters to mosaic dataset")

# Add time field
arcpy.AddField_management(mosaic_path, "date", "DATE")

# Calculate time from filename
with arcpy.da.UpdateCursor(mosaic_path, ["Name", "date"]) as cursor:
    for row in cursor:
        filename = row[0]
        # Remove .tif extension and split
        parts = filename.replace(".tif", "").split("_")

        # Join last two parts to get full timestamp: 20240926 + 080000
        time_str = parts[-2] + parts[-1]  # Combines date and time

        # Parse: 20240926080000 -> datetime
        dt = datetime.strptime(time_str, "%Y%m%d%H%M%S")
        print(f"{filename} -> {time_str} -> {dt}")
        row[1] = dt
        cursor.updateRow(row)

print("Time field populated")

# Configure mosaic properties
arcpy.SetMosaicDatasetProperties_management(
    mosaic_path,
    start_time_field="date"
)

print("Mosaic dataset configured with time dimension")

print(f"\nMosaic dataset complete: {mosaic_path}")
print("To apply symbology in ArcGIS Pro:")
print(f"1. Add mosaic to map: {mosaic_path}")
print(f"2. Right-click layer > Symbology > Import")
print(f"3. Select: {symbology_file}")
print("4. Enable time slider to animate cumulative growth")

In [ ]:
# ==============================================================================
# SPACE-TIME CUBE GENERATION (NetCDF Format)
# Creates 3D cubes from raster time series without ArcPy
# ==============================================================================

import os
import numpy as np
import rasterio
from netCDF4 import Dataset
from datetime import datetime
import glob

def create_space_time_cube(raster_folder, output_nc_path, hurricane_name, cube_type):
    """
    Convert a folder of time-sequenced rasters into a space-time cube (netCDF).

    Parameters:
    -----------
    raster_folder : str
        Path to folder containing GeoTIFF rasters
    output_nc_path : str
        Output path for netCDF file
    hurricane_name : str
        Name of hurricane (e.g., 'helene')
    cube_type : str
        Type of cube ('increment' or 'cumulative')
    """
    print(f"\nCreating space-time cube for {hurricane_name} ({cube_type})...")

    # Get all raster files sorted by timestamp
    raster_files = sorted(glob.glob(os.path.join(raster_folder, '*.tif')))

    if len(raster_files) == 0:
        print(f"ERROR: No rasters found in {raster_folder}")
        return

    print(f"  Found {len(raster_files)} rasters")

    # Read first raster to get dimensions and metadata
    with rasterio.open(raster_files[0]) as src:
        height = src.height
        width = src.width
        transform = src.transform
        crs = src.crs

    print(f"  Grid dimensions: {width} x {height}")

    # Parse timestamps from filenames
    timestamps = []
    for f in raster_files:
        # Extract timestamp from filename: hurricane_tweets_YYYYMMDD_HHMMSS.tif
        basename = os.path.basename(f)
        parts = basename.replace('.tif', '').split('_')
        time_str = parts[-2] + parts[-1]  # YYYYMMDDHHMMSS
        dt = datetime.strptime(time_str, '%Y%m%d%H%M%S')
        timestamps.append(dt)

    # Create netCDF file
    print(f"  Creating netCDF file: {output_nc_path}")
    nc = Dataset(output_nc_path, 'w', format='NETCDF4')

    # Create dimensions
    nc.createDimension('x', width)
    nc.createDimension('y', height)
    nc.createDimension('time', len(raster_files))

    # Create coordinate variables
    x_var = nc.createVariable('x', 'f8', ('x',))
    y_var = nc.createVariable('y', 'f8', ('y',))
    time_var = nc.createVariable('time', 'f8', ('time',))

    # Create data variable
    data_var = nc.createVariable('tweet_density', 'f4', ('time', 'y', 'x'),
                                   fill_value=-9999, zlib=True, complevel=4)

    # Set coordinate values
    x_coords = np.arange(width) * transform.a + transform.c  # X coordinates
    y_coords = np.arange(height) * transform.e + transform.f  # Y coordinates
    x_var[:] = x_coords
    y_var[:] = y_coords

    # Convert timestamps to numeric (days since epoch)
    epoch = datetime(1970, 1, 1)
    time_values = [(t - epoch).total_seconds() / 86400.0 for t in timestamps]
    time_var[:] = time_values

    # Add metadata
    nc.title = f'{hurricane_name.capitalize()} Hurricane Tweet Activity ({cube_type})'
    nc.institution = 'Generated from tweet geolocation data'
    nc.source = 'Twitter/X social media data'
    nc.hurricane = hurricane_name
    nc.cube_type = cube_type
    nc.Conventions = 'CF-1.6'

    # Variable metadata
    x_var.units = 'meters'
    x_var.long_name = 'x coordinate (EPSG:3857)'
    x_var.standard_name = 'projection_x_coordinate'

    y_var.units = 'meters'
    y_var.long_name = 'y coordinate (EPSG:3857)'
    y_var.standard_name = 'projection_y_coordinate'

    time_var.units = 'days since 1970-01-01 00:00:00'
    time_var.long_name = 'time'
    time_var.standard_name = 'time'
    time_var.calendar = 'gregorian'

    data_var.units = 'tweet_activity_index'
    data_var.long_name = 'Tweet Activity Density'
    data_var.description = f'{cube_type.capitalize()} tweet activity from hierarchical rasterization (states, counties, cities KDE, facilities)'

    # Load and write raster data
    print(f"  Loading raster data...")
    for i, raster_path in enumerate(raster_files):
        with rasterio.open(raster_path) as src:
            data = src.read(1)  # Read first band
            data_var[i, :, :] = data

        if (i + 1) % 5 == 0:
            print(f"    Loaded {i + 1}/{len(raster_files)} time slices")

    # Close netCDF file
    nc.close()

    print(f"  ✓ Space-time cube created successfully!")
    print(f"    Dimensions: {width} x {height} x {len(raster_files)} (X x Y x Time)")
    print(f"    Time range: {timestamps[0]} to {timestamps[-1]}")
    print(f"    Output: {output_nc_path}")


# ==============================================================================
# GENERATE SPACE-TIME CUBES FOR ALL DATASETS
# ==============================================================================

print("\n" + "=" * 60)
print("GENERATING SPACE-TIME CUBES")
print("=" * 60)

# Create output directory for cubes
cubes_dir = os.path.join(local_path, 'space_time_cubes')
os.makedirs(cubes_dir, exist_ok=True)

# Generate cubes for each hurricane and type
datasets = [
    ('helene', 'cumulative'),
    ('helene', 'increment'),
    ('francine', 'cumulative'),
    ('francine', 'increment')
]

for hurricane, cube_type in datasets:
    raster_folder = os.path.join(output_dir, hurricane, cube_type)
    output_nc = os.path.join(cubes_dir, f'{hurricane}_{cube_type}_cube.nc')

    if os.path.exists(raster_folder):
        create_space_time_cube(raster_folder, output_nc, hurricane, cube_type)
    else:
        print(f"\nWARNING: Folder not found: {raster_folder}")

print("\n" + "=" * 60)
print("SPACE-TIME CUBE GENERATION COMPLETE!")
print("=" * 60)
print(f"\nOutput directory: {cubes_dir}")
print("\nYou can now:")
print("  1. Open these .nc files in ArcGIS Pro as multidimensional rasters")
print("  2. Use Python libraries like xarray to analyze them")
print("  3. Visualize with tools like Panoply or ncview")
